# Finding optimal parameters
With the setup of all required function, I explore optimal parameter values in the test set. Note that for robust parameter values, more comprenhesive testing should be done.

In [5]:
import pandas as pd
import pandas_datareader as pdr 
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import networkx as nx
import datetime
from scipy.stats import pearsonr
from trading_functions import *

In [90]:
# Load ticker symbols from a CSV file into a DataFrame.
ticker_df = pd.read_csv('nasdaq_screener_1686671592339.csv')

# Extract the 'Symbol' column from the DataFrame and convert it to a list of ticker symbols.
tickers = list(ticker_df['Symbol'].values)

# Limit the list of tickers to the first 500 for manageability or computational efficiency.
tickers = tickers[0:500]

# Define the start and end dates for the training period of the trading strategy.
training_start_date = "2010-01-03"
training_end_date = "2021-01-01"

# Define the start and end dates for the first testing period of the trading strategy.
testing_start_date = '2021-01-03'
testing_end_date = '2023-04-01'


In [ ]:
# Set the threshold for determining if a link (correlation) between assets is strong enough to consider.
link_threshold = 0.6

# Set the threshold for how frequently a link must occur to be considered significant.
frequency_threshold = 0.5

# Set the starting capital for the investment strategy.
initial_capital = 1000000

# Set the parameter determining how much to invest based on the strategy's signals.
investment_parameter = 0.2

# Set the threshold for determining if a loss in link (correlation) strength between assets is significant.
link_loss_threshold = 0

# Set the threshold for determining when to drop an asset pair based on decreased correlation.
drop_out_threshold = 0.8

# A boolean flag to control whether to draw plots for visual analysis. False means no plots will be drawn.
draw = False

# Set the step size for iterating through ranges of parameters when optimizing the strategy.
step = 0.5

# Set the annual risk-free rate, used for calculating metrics like the Sharpe Ratio.
annual_risk_free_rate = 0.005

# Retrieve price and daily returns data for training period using a predefined list of tickers and date range.
price_df_train, daily_returns_df_train = get_data(tickers, training_start_date, training_end_date)

# Retrieve price and daily returns data for testing period using the same list of tickers and a different date range.
price_df_test, daily_returns_df_test = get_data(tickers, testing_start_date, testing_end_date)

# Set the number of time windows to split the data into for analyzing correlations and performance.
n_time_windows = 100

# A boolean flag to indicate whether the size of the time windows is fixed (True) or should be calculated dynamically (False).
size_boolean = True

# If time window sizes are fixed, this sets the size of each time window.
chosen_time_window_size = 30

# Calculate the total number of days in the training dataset.
tot_days = len(price_df_train)

# Split the total days into time windows based on the number of windows and size settings defined above.
time_window_splits, time_window_size = split_time_windows(tot_days, n_time_windows, chosen_time_window_size, size_boolean)


In [ ]:
# Set the step size for iterating through parameter values.
step = 0.1

# Initialize an empty dictionary to store results of metric calculations.
metric_results_dict = {}

# Nested loops to iterate through different thresholds and parameters in increments of 'step'.
for link_threshold in np.arange(0, 1 + step, step):
    for frequency_threshold in np.arange(0, 1 + step, step):
        for drop_out_threshold in np.arange(0, 1 + step, step):
            for investment_parameter in np.arange(0, 1 + step, step):
                # Print the current set of parameters for debugging or tracking progress.
                print((link_threshold, frequency_threshold, drop_out_threshold, investment_parameter))

                # Find groups of correlated assets based on the current link_threshold and frequency_threshold.
                separate_groups = find_correlation_groups(time_window_splits, price_df_train, link_threshold, frequency_threshold, draw)

                # Identify pairs of assets to drop based on the drop_out_threshold in the test dataset.
                correlation_pairs_test_dict, pairs_to check = find_dropout_candidates(price_df_test, time_window_splits_test, drop_out_threshold, separate_groups)

                # Choose which of the identified dropout candidates to actually drop.
                drop_out_chosen_dict = choose_dropout_candidates(price_df_test, correlation_pairs_test_dict, separate_groups, drop_out_threshold, pairs_to_check)

                # Generate trading signals based on the current set of parameters.
                signals_dictionary, current_capital = create_signals(price_df_test, time_window_splits_test, drop_out_threshold, link_loss_threshold, initial_capital, investment_parameter, separate_groups)

                # Create a portfolio based on the generated signals and calculate its performance over the test period.
                balance_df = create_portfolio(signals_dictionary, testing_start_date, testing_end_date, initial_capital)

                # Calculate performance metrics for the strategy with the current set of parameters and store the results.
                metric_results_dict[(link_threshold, frequency_threshold, drop_out_threshold, investment_parameter)] = strategy_metrics(balance_df, annual_risk_free_rate)


In [300]:
# Initialize a dictionary to store aggregated results.
results_dict = {'parameters':[], 'total_return':[], 'total_return_vs_risk_free':[], 'sharpe_ratio':[]}

# Iterate over each set of parameters and corresponding results in the metric_results_dict.
for parameters, one_result_dict in metric_results_dict.items():
    # Append the parameters and calculated metrics (total return, total return vs. risk-free rate, Sharpe ratio) to the results_dict.
    results_dict['parameters'].append(parameters)
    results_dict['total_return'].append(one_result_dict['total_return'])
    results_dict['total_return_vs_risk_free'].append(one_result_dict['total_return_vs_risk_free'])
    results_dict['sharpe_ratio'].append(one_result_dict['sharpe_ratio'])

# Convert the results_dict to a DataFrame for easier data manipulation.
results_df = pd.DataFrame(results_dict)

# Sort the DataFrame by total return in descending order and reset the index.
results_df = results_df.sort_values(by='total_return', ascending=False).reset_index(drop=True)

# Reset the index again to use it as a ranking column for total returns (rank_1).
results_df = results_df.reset_index()
results_df = results_df.rename(columns={"index": "rank_1"})

# Sort the DataFrame by Sharpe ratio in descending order and reset the index.
results_df = results_df.sort_values(by='sharpe_ratio', ascending=False).reset_index(drop=True)

# Reset the index again to use it as a ranking column for Sharpe ratios (rank_2).
results_df = results_df.reset_index()
results_df = results_df.rename(columns={"index": "rank_2"})

# Calculate the average rank based on the two ranking columns created above.
results_df['average_rank'] = (results_df['rank_1'] + results_df['rank_2']) / 2

# Sort the DataFrame by the average rank to determine the overall performance taking into account both total return and Sharpe ratio.
results_df = results_df.sort_values(by='average_rank')

# Return the sorted DataFrame with all calculated columns and rankings.
results_df


,rank_2,rank_1,parameters,total_return,total_return_vs_risk_free,sharpe_ratio,average_rank
0,0,0,"(0.0, 0.0, 0.0, 1.0)",0.001250,-0.009893,-0.033569,0.0
1,1,1,"(0.0, 0.0, 0.0, 0.9)",0.001124,-0.010017,-0.037838,1.0
2,2,2,"(0.0, 0.0, 0.0, 0.8)",0.001000,-0.010140,-0.043143,2.0
3,3,3,"(0.0, 0.0, 0.0, 0.7000000000000001)",0.000875,-0.010264,-0.049991,3.0
4,4,4,"(0.0, 0.0, 0.0, 0.6000000000000001)",0.000750,-0.010388,-0.059120,4.0
5,5,5,"(0.0, 0.0, 0.0, 0.5)",0.000624,-0.010511,-0.071902,5.0
6,6,6,"(0.0, 0.0, 0.0, 0.4)",0.000500,-0.010634,-0.090967,6.0
7,7,7,"(0.0, 0.0, 0.0, 0.30000000000000004)",0.000375,-0.010758,-0.122876,7.0
8,8,8,"(0.0, 0.0, 0.0, 0.2)",0.000250,-0.010882,-0.186756,8.0
9,9,9,"(0.0, 0.0, 0.0, 0.1)",0.000125,-0.011006,-0.378977,9.0
